# Ransac affine for point cloud alignment of two masks
In this tutorial we will learn to warp moving masks and examine point clouds (centriod of ROI) using two variants of the ransac affine and ICP algorithm. We can apply the ROI to extract images for later ROI_affine

Preprocessing for masks of moving rounds:
1. 3d starfinity segmentation of fixed round DAPI (segmentation masks)
2. filter ROI (remove non-neuron edge masks, merge over segmentation masks）
3. apply warp/invtransform to fixed round DAPI mask
4. remove noise on edge, mannually check the ROI shape

Steps for compare 2 rounds of masks, and export ROI_SPOTS
1. Extract ROI centriods as point cloud. Calculate the knn distance (20 pixels)
2. Ransac affine the point sets (<10% outlier, distance is 20 pixels). 
    a. spot extraction, context(no images any more), remove outlier by calculating correlation, ransac affine
    b. may add other information for the ROI_matching: # relative positions (distance to 3 nearest points), area (within 0.5-1.5), aspect_ratio.
    b.Visualize the masks and points
3. Use the ICP to minimize the distance of correponding ROIs. visualize the masks and points.Find the nearest ROIs using knn-neighbor. 

Bash
#maybe not necessary

#### input: 
1)segmentation mask of fixed rounds, 2) invtransformation, 3)extracted spots, 4)ROI_intensity info
#### output: 
1) segmentation mask of moving rounds, 2)point clouds of masks(center of each mask), 3) ROI index and meata,4) ROI_spots; 5) distance of warped ROI and real ROI (from the segmentation). 5. Generate excel for corresponding ROIs. 


We begin with loading the required modules.

In [1]:
# import pools
import os
import sys
import itertools
from math import pi, sin, cos, sqrt
import matplotlib.pyplot as plt
import numpy as np
import cv2
from matplotlib import cm
from scipy import ndimage
import scipy.io
from skimage import data
from skimage.io import imread, imsave
import pandas as pd
from scipy.spatial import cKDTree
from cv2 import estimateTranslation3D
import tifffile
import seaborn as sns
from skimage.measure import regionprops
from scipy.spatial import distance

# import bigstream library
import zarr
from bigstream import ransac
from bigstream import transform

# napari
%gui qt5
import napari
# viewer = napari.view_image(data.astronaut(), rgb=True)
# napari.run()

We begin with loading the required modules for ransac.

In [2]:
def ransac_align_points(
    pA, pB, threshold, diagonal_constraint=0.75, default=np.eye(4)[:3],
):
    """
    """

    # sensible requirement of 51 or more spots to compute ransac affine
    if len(pA) <= 10 or len(pB) <= 10:
        if default is not None:
            print("Insufficient spot matches for ransac, returning default identity")
            return default
        else:
            raise ValueError("Insufficient spot matches for ransac, need more than 10")

    # compute the affine
    r, Aff, inline = cv2.estimateAffine3D(pA, pB, ransacThreshold=threshold, confidence=0.999)

    print(np.diag(Aff))
    print(Aff)
    print(diagonal_constraint)
    # rarely ransac just doesn't work (depends on data and parameters)
    # sensible choices for hard constraints on the affine matrix
    if np.any( np.diag(Aff) < diagonal_constraint ):
        if default is not None:
            print("Degenerate affine produced, returning default identity")
            return default
        else:
            raise ValueError("Degenerate affine produced, ransac failed")

    return Aff

### Colocalization filter 

In [7]:
def colocalization(spot_c0,spot_c1,neighbor_radius):
    # Fix and Mov true spots
#     neighbor_radius = math.sqrt(3*3*3) #3.464
    #vox=[0.23,0.23,0.38]
    c0=spot_c0[:,:3].copy()
    c1=spot_c1[:,:3].copy()

    kdtree_c0 = cKDTree(c0)
    kdtree_c1 = cKDTree(c1)
    neighbors = kdtree_c0.query_ball_tree(kdtree_c1, neighbor_radius)
    # neighbors1 = kdtree_c0.query_ball_point(c1,neighbor_radius) #1 is all unique idx, 2 is the location
    # neighbors2 = kdtree_c0.sparse_distance_matrix(kdtree_c1, neighbor_radius)
    dist2,idx2 = kdtree_c0.query(c1, k=3)
    [Idx_unique, I] = np.unique(idx2,return_index=True) 
#     print(len(dist2[:,0])*3)
    
    ## find the smallest repeated value location, and delete the others
    for i in range(Idx_unique.shape[0]):  # should repeat for only once
    #     print(Idx_unique[i])
        Loc_rep=np.where(idx2==Idx_unique[i])    
    #     print(Loc_rep[0])
        A=dist2[Loc_rep[0],Loc_rep[1]]
        minposition = min(A)
        Loc_min = np.where(A==minposition)[0]
    #     print(Loc_min)
    #     Loc_rep_min=Loc_rep[0][Loc_min[0]]    
        Loc_rep_nouse=np.delete(range(len(Loc_rep[0])),Loc_min)
        dist2[Loc_rep[0][Loc_rep_nouse],Loc_rep[1][Loc_rep_nouse]]=neighbor_radius*2 
    ## find the results that are less than radius; used later column data 
    # when only first row is not exist use latter column, or just dispose it. 
    co_loc=np.where(dist2>neighbor_radius)

    for j in range(dist2.shape[0]):
         if dist2[j,0] < neighbor_radius:
                dist2[j,1] = neighbor_radius*2
    for j in range(dist2.shape[0]):            
         if dist2[j,0] <neighbor_radius or dist2[j,1] <neighbor_radius:
                dist2[j,2] = neighbor_radius*2       
    row_c1 = np.where(dist2<neighbor_radius)
#     print(len(row_c1[0]))

    # lipo spot_c1 is row_c1
    pBind = row_c1[0]
    # print(idx2)
    # lipo spot_c1 is idx2
    pAind = [(idx2[row_c1[0][x], row_c1[1][x]]) for x in range(len(row_c1[0]))]
    lipo_c0 = spot_c0[pAind]
    lipo_c1 = spot_c1[pBind]

#     print(np.unique(pAind).shape)
    
    true_pos_c0 = np.delete(spot_c0, pAind, axis=0)
    true_pos_c1 = np.delete(spot_c1, pBind, axis=0) #true

    if spot_c0.shape[0]>0:
        P1 = (lipo_c0.shape[0] / spot_c0.shape[0])*100   # % mov spots from  previous images  /spot_c0.shape
    else:
        P1 = 0
        
    if spot_c1.shape[0]>0:
        P2 = (lipo_c1.shape[0] / spot_c1.shape[0])*100  # % fixed spots can be found in later mov images  /spot_c0.shape
    else:
        P2 = 0

    print('% P1: ',str(P1) + ';  % P2: ',str(P2)) 

    Dist = eucldist(lipo_c0,lipo_c1)

    return lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,Dist

# compute distance of paired spot cloud
def cloud_distance(spot_fix,spot_mov):
    """
    """
    c0=spot_fix[:,:3].copy()
    c1=spot_mov[:,:3].copy()
    kdtree_c0 = cKDTree(c0)
    kdtree_c1 = cKDTree(c1)
    dist2,idx2 = kdtree_c0.query(c1, k=3)
    [Idx_unique, I] = np.unique(idx2,return_index=True) 
    return dist2[:,0]

# euclidean_distances(lipo_c0,lipo_c1).shape
from sklearn.metrics.pairwise import euclidean_distances
def eucldist(coords1, coords2):
    """ Calculates the euclidean distance between 2 lists of coordinates. """
    dist = np.zeros(len(coords1))
    i = 0
    for (x, y) in zip(coords1, coords2):
        p1 = x
        p2 = y
        squared_dist = (p1[0]-p2[0])**2+(p1[1]-p2[1])**2+(p1[2]-p2[2])**2
        dist[i] = np.sqrt(squared_dist)
        i = i+1
    return dist
# dist = eucldist(lipo_c0, lipo_c1)
# print(dist)

def match_points(A, B, scores, threshold):
    """
    """
    scores = 0
    A1,B1,_,_,_ = colocalization(A,B,threshold)
    # return positions of corresponding points
    return A1,B1

def violin_distance(A, B):    
    fig=plt.figure(dpi=120,figsize=(2,3))
    plt.violinplot(cloud_distance(A,B))
    sns.despine() 
    plt.xlabel('Spots:'+ str(cloud_distance(A,B).shape[0]))
    plt.ylabel('Distance/pixel')
    ave=np.average(cloud_distance(A,B))
    plt.title(str(float('%.2f' % ave)))
    plt.show()
    plt.tight_layout()
    return ave

def pair_match(spot_c0,spot_c1,neighbor_radius):
    c0=spot_c0[:,:3].copy()
    c1=spot_c1[:,:3].copy()
    kdtree_c0 = cKDTree(c0)
    kdtree_c1 = cKDTree(c1)
    neighbors = kdtree_c0.query_ball_tree(kdtree_c1, neighbor_radius)
    dist2,idx2 = kdtree_c0.query(c1, k=3)
    [Idx_unique, I] = np.unique(idx2,return_index=True) 
    ## find the smallest repeated value location, and delete the others
    for i in range(Idx_unique.shape[0]):  # should repeat only once
        Loc_rep=np.where(idx2==Idx_unique[i])    
        A=dist2[Loc_rep[0],Loc_rep[1]]
        minposition = min(A)
        Loc_min = np.where(A==minposition)[0]
        Loc_rep_nouse=np.delete(range(len(Loc_rep[0])),Loc_min)
        dist2[Loc_rep[0][Loc_rep_nouse],Loc_rep[1][Loc_rep_nouse]]=neighbor_radius*2 
    ## find the results that are less than radius; used later column data 
    # use latter column when only first row is not exist, or just dispose it. 
    co_loc=np.where(dist2>neighbor_radius)

    for j in range(dist2.shape[0]):
         if dist2[j,0] < neighbor_radius:
                dist2[j,1] = neighbor_radius*2
    for j in range(dist2.shape[0]):            
         if dist2[j,0] <neighbor_radius or dist2[j,1] <neighbor_radius:
                dist2[j,2] = neighbor_radius*2       
    row_c1 = np.where(dist2<neighbor_radius)
    pBind = row_c1[0]
    pAind = [(idx2[row_c1[0][x], row_c1[1][x]]) for x in range(len(row_c1[0]))]
#     lipo_c0 = spot_c0[pAind]
#     lipo_c1 = spot_c1[pBind]
    Aind = spot_c0[pAind][:,3].astype(int)
    Bind = spot_c1[pBind][:,3].astype(int)
    return Aind,Bind

In [10]:
## ICP
def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nx3 array of points
        dst: Nx3 array of points
    Output:
        distances: Euclidean distances (errors) of the nearest neighbor
        indecies: dst indecies of the nearest neighbor
    '''

    indecies = np.zeros(src.shape[0], dtype=np.int)
    distances = np.zeros(src.shape[0])
    for i, s in enumerate(src):
        min_dist = np.inf
        for j, d in enumerate(dst):
            dist = np.linalg.norm(s-d)
            if dist < min_dist:
                min_dist = dist
                indecies[i] = j
                distances[i] = dist
    return distances, indecies  

def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform between corresponding 3D points A->B
    Input:
      A: Nx3 numpy array of corresponding 3D points
      B: Nx3 numpy array of corresponding 3D points
    Returns:
      T: 4x4 homogeneous transformation matrix
      R: 3x3 rotation matrix
      t: 3x1 column vector
    '''
    assert len(A) == len(B)

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0) 
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    W = np.dot(BB.T, AA)
    U, s, VT = np.linalg.svd(W, full_matrices=True, compute_uv=True)
    R = np.dot(U, VT)

    # special reflection case
    if np.linalg.det(R) < 0:
        VT[2,:] *= -1
        R = np.dot(U, VT)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = t

    return T, R, t
       
def icp(A, B, ICP_error,init_pose = None, max_iterations=200, tolerance=0.001):
    '''
    The Iterative Closest Point method
    Input:
        A: Nx3 numpy array of source 3D points
        B: Nx3 numpy array of destination 3D point
        init_pose: 4x4 homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation
        distances: Euclidean distances (errors) of the nearest neighbor
    '''
    #  select points 
#     ICP_error = 30
    A,B,_,_,_ = colocalization(A,B,ICP_error)
    
    # make points homogeneous, copy them so as to maintain the originals
    src = np.ones((4,A.shape[0]))  #(4, A.shape[0])
    dst = np.ones((4,B.shape[0]))
    src[0:3,:] = np.copy(A.T)  # A.T shape (3,20)
    dst[0:3,:] = np.copy(B.T)
    
    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0
    distances_iter = np.zeros((max_iterations,1))

    for i in range(max_iterations):
        # find the nearest neighbours between the current source and destination points
        distances, indices = nearest_neighbor(src[0:3,:].T, dst[0:3,:].T)
        
        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[0:3,:].T, dst[0:3,indices].T)  
        # update the current source
    # refer to "Introduction to Robotics" Chapter2 P28. Spatial description and transformations
        src = np.dot(T, src)

        # check error
        mean_error = np.sum(distances) / distances.size
#         print(f'mean_error: {mean_error}')
#         if abs(prev_error-mean_error) < tolerance:
#             break
        prev_error = mean_error
        distances_iter[i] = mean_error
    
    fig=plt.figure(dpi=120,figsize=(2,3))
    plt.plot(distances_iter[:20])
    sns.despine() 
    plt.xlabel('Spots:'+ str(distances_iter[:20].shape[0]))
    plt.ylabel('Distance/pixel')
    ave=np.average(distances_iter[:20])
    plt.title(str(float('%.2f' % ave)))
    plt.show()
    # calculcate final tranformation
    T,_,_ = best_fit_transform(A, src[0:3,:].T)

    return T, distances     

In [4]:
####Get metadata for ROI: ID, centroid position, size, distance to (0,0,0) and aspect ratio####
def ROI_meta(segmentation,out_dir):
    lb = segmentation.astype(int)
    roi = np.max(lb)
    lb_id = np.unique(lb[lb != 0]) 
    df = pd.DataFrame(np.empty([lb_id.shape[0], 0]))
    lb_stat = regionprops(lb)
    ROI_points = np.empty([lb_id.shape[0],4])
    i = 0
    for j in lb_id: 
        df.loc[df.index[i], 'roi'] = j
        df.loc[df.index[i], 'x'] = lb_stat[i].centroid[0]
        df.loc[df.index[i], 'y'] = lb_stat[i].centroid[1]
        df.loc[df.index[i], 'z'] = lb_stat[i].centroid[2]
        df.loc[df.index[i], 'area'] = lb_stat[i].area
#         df.loc[df.index[i], 'Distance'] = distance.euclidean(lb_stat[i].centroid,[0,0,0])
        df.loc[df.index[i], 'minor_axis_length'] = lb_stat[i].minor_axis_length
        df.loc[df.index[i], 'major_axis_length'] = lb_stat[i].major_axis_length
        df.loc[df.index[i], 'aspect_ratio'] = lb_stat[i].minor_axis_length/lb_stat[i].major_axis_length
        ROI_points[i,:3] = lb_stat[i].centroid
        ROI_points[i,3] = j
        i = i+1
    ####Filter out ROIs that 1) not in mask; 2) have high background in channel 546
    list_ROI = [0]
    df_filtered=df.loc[~df['roi'].isin(list_ROI)]
    df_filtered.to_csv(out_dir, index=False)
    print(ROI_points.shape)
    return df_filtered,ROI_points

In [ ]:
# 1) correlation (corr > 0.999 in average intensity of 4 channels) 
# 2) less than 75 pixels apart(centroid position distance, for 3x expansion)
# 3) > 4% of cell volume contacting
# 4) for ROIs that have more than two corresponding pairs, the corresponding segments will be ranked by % of contacting, 
# and they will be flagged for manual inspection
# 5) if both segments are bigger than 20000 in size, it will be flagged and manually inspected
# 6) at least one of the segments have to be bigger than 7000 pixels in size

In [6]:
def downsegmentation_ROI(lb, r, out_dir, count_dir, Mean_int_dir, roi_dir):

# Would need to test if it is the same with Yuhan's orginal one with multiple rounds 

# Code used to flag over-segmentation errors for a single round. 
# A. maximize the number of identified errors
# B. minimize false detection of well-segemented cells 
# 
# The over-segmentation pairs are identified with the following criteria: 
# 1) the two segments have a very high correlation (corr > 0.999 in average intensity of 4 channels) 
# 2) the two segments have to be less than 75 pixels apart(centroid position distance, for 3x expansion)
# 3) The two segments have to be touching each other: at least one > 4% of cell volume contacting
# 4) for ROIs that have more than two corresponding pairs, the corresponding segments will be ranked by % of contacting, 
# and they will be flagged for manual inspection
# 5) if both segments are bigger than 20000 in size, it will be flagged and manually inspected
# 6) at least one of the segments have to be bigger than 7000 pixels in size
# #1，#2, #3, #4 are for initial identification of oversegmentation errors 
# #5, #6 are for eliminating the false detections  

#     count_dir      = seg_dir + 'R2_3tm50_1920/roi_spots_all.csv' # directory to spot count/neuron (csv format)
#     Mean_int_dir   = seg_dir + 'R2_mean_intensity.csv' # directory to mean intensity/neuron (csv format)
# #     roi_dir        = seg_dir + 'allroi_fix.csv'
#     lb_dir         = 'E:/Maxprobe_analysis/R2_R1_3tm50/mask2.tif' # directory to segmentation mask (tif format)
#     out_dir        = seg_dir

    df1=pd.read_csv(count_dir,sep=',', index_col=0) ## / change to count/area so that it can be comparable.
    df2=pd.read_csv(Mean_int_dir,sep=',', index_col=0)
    # df= pd.concat([df1, df2], axis=1)
    df = df2
    roi=pd.read_csv(roi_dir,sep=',',index_col=0)
    # roi=df_filtered_fix

    ###Get correlation matrix
    corr_raw =df.T.corr()
    s_raw = corr_raw.stack()
    ii_raw = s_raw[np.logical_and(s_raw > 0.999, s_raw<1.0)].index.tolist()
    test=np.asarray(ii_raw)
    test.sort(axis=1)
    test=np.unique(test,axis=0).astype(int)

    cand={}
    for i in range(0,len(test)):
        a=roi.loc[test[i,0]].to_numpy()[:3]
        b=roi.loc[test[i,1]].to_numpy()[:3]
        dist=distance.euclidean(a,b)
        if dist<75 and dist>0:  # for 3x expansion
            a_area=roi.loc[test[i,0]]['area']
            b_area=roi.loc[test[i,1]]['area']
            if np.maximum(a_area,b_area) >7000:
                c=corr_raw.loc[test[i,0],test[i,1]]
                cand[i] = np.append(test[i,:],c)
                cand[i] = np.append(cand[i],dist)
                if np.minimum(a_area,b_area) >20000:
                    cand[i]=np.append(cand[i], str(np.minimum(a_area,b_area)))
                else:
                    cand[i]=np.append(cand[i], str('--'))

    m=pd.DataFrame.from_dict(data=cand, orient='index')
    m = m.rename(columns={0:'cell_A', 1:'cell_B', 2: 'corr', 3:'dist',4: 'min_size_20000'})
    m['cell_A'], m['cell_B'] = np.where(m['cell_A'] > m['cell_B'], [m['cell_B'],m['cell_A'] ], [m['cell_A'] , m['cell_B']])

    ### % of cell volume contacting
#     lb=imread(lb_dir)
    lb_stat=regionprops(lb)
    select={}            
    ROI_row =[]
    ROI_row_1 =[]
    ROI_row_2 =[]
    ROI_row_3 =[]
    ROI_row_4 =[]

    kk = 0
    ROI_merge = {}
    ROI_remove = []
    for k in range(0,len(m)):
        id1=m.iloc[k]['cell_A'].astype(np.float).copy()
        id2=m.iloc[k]['cell_B'].astype(np.float).copy()
        a=lb_stat[int(id1-1)].coords
        b=lb_stat[int(id2-1)].coords
        kdtree_a = cKDTree(a)
        kdtree_b = cKDTree(b)
        nnn=kdtree_a.count_neighbors(kdtree_b,1)    
        edge_ratio_a = 100*nnn/roi.loc[id1]['area'].astype(int)
        edge_ratio_b = 100*nnn/roi.loc[id2]['area'].astype(int)

        if nnn>0:
            select[k]=m.iloc[k]
            # edge_ratio_a
            if edge_ratio_a<4:
    #             select[k]=np.append(select[k], str('less_100'))
                select[k]=np.append(select[k], str('less_4%')) 
            else:
                select[k]=np.append(select[k], str(edge_ratio_a)) # ('--')
            # edge_ratio_b    
            if edge_ratio_b<4:
    #             select[k]=np.append(select[k], str('less_100'))
                select[k]=np.append(select[k], str('less_4%')) 
            else:
                select[k]=np.append(select[k], str(edge_ratio_b)) # ('--')

            if (edge_ratio_b < 4) and (edge_ratio_a < 4):    
                select[k]=np.append(select[k], str('--')) # ('--')
            else:
                # get all rows
                ROI_merge1 = [int(id1),int(id2),edge_ratio_a,edge_ratio_b]
                ROI_merge[kk] = ROI_merge1 # ('--')
                kk = kk + 1
                # 'cell_B' can be only merge to one 'cell_A': find the biggest edge_ratio_b;      
                select[k]=np.append(select[k], str('merge'))
                # 'cell_A' can be only merge to one 'cell_B': find the biggest edge_ratio_a;  

    ### generate oversegmentation list
    select=pd.DataFrame.from_dict(data=select, orient='index')
    select = select.rename(columns={0:'cell_A', 1:'cell_B', 2: 'corr', 3:'dist',4: 'min_size_20000',5:'touch_A',6:'touch_B',7:'Merge'})
    select.to_csv(out_dir + '/'+ r + '_oversegmentation.csv')

    ### automatic merge oversegmenation masks when touching ratio > 4%
    n=pd.DataFrame.from_dict(data=ROI_merge, orient='index')
    n = n.rename(columns={0:'cell_A', 1:'cell_B', 2: 'touch_A', 3:'touch_B'})
    for j in range(0,kk):    
        # 'cell_A' can be only merge with one 'cell_B': find the biggest edge_ratio_a; 
        ROI_merge_B = n.iloc[j]['cell_B'].astype(np.int32).copy()
        ROI_row = list(np.where(n['cell_B'] == ROI_merge_B))[0]
        if ROI_row.shape[0] > 1:
            ROI_row_1 = list(np.where(ROI_row != n['touch_A'][ROI_row].idxmax())[0])
    #         print(list(ROI_row[ROI_row_1]))
            ROI_row_2 = np.array(list(set(ROI_row_2).union(set(list(ROI_row[ROI_row_1])))))

        # 'cell_B' can be only merge to'cell_A' once: find the biggest edge_ratio_b;
        ROI_merge_A = n.iloc[j]['cell_A'].astype(np.int32).copy()
        ROI_row = list(np.where(n['cell_A'] == ROI_merge_A))[0]
        if ROI_row.shape[0] > 1:
            ROI_row_3 = list(np.where(ROI_row != n['touch_B'][ROI_row].idxmax())[0])
            ROI_row_4 = np.array(list(set(ROI_row_4).union(set(list(ROI_row[ROI_row_3])))))
        # delete select row.
    ROI_remove  = list(np.array(set(ROI_row_4).union(set(ROI_row_2))).tolist())
    ROI_keep = list(np.array(set(range(kk)).difference(set(ROI_remove))).tolist())
    ROI_select = [n.iloc[i][['cell_A','cell_B','touch_A','touch_B']] for i in list(ROI_keep)]
    ROI_select=pd.DataFrame(ROI_select) 
    ROI_select.to_csv(out_dir + '/'+ r + '_oversegmentation_final.csv')  
    return ROI_select

def merge_ROI(lb, ROI_select):
    # Merge list
    A = np.array(ROI_select.values.tolist())[:,0].astype(int) # cell_A
    B = np.array(ROI_select.values.tolist())[:,1].astype(int) # cell_B
    C = list(np.array(set(A).intersection(set(B))).tolist()) # replicate list
    row_C = [list(np.where(A == i)[0])[0] for i in C]  
    lb_vary = lb.copy()
    for i in C: #Merge replicate roi first        
        row_A = A[np.where(A == i)]
        row_B = B[np.where(A == i)] # find the row 
        # merge row_C = i
        lb_vary[np.where(lb_vary==row_B)] = row_A
        
    # remove the row_B from list
    A = np.delete(A, row_C, axis=0) 
    B = np.delete(B, row_C, axis=0) 
    ii = 0
    for j in A :   
    # merge A and B
        lb_vary[np.where(lb_vary==B[ii])] = j
        ii = ii +1
    return(lb_vary)

In [58]:
def integrated_intensity(regionmask, intensity_image):
    return np.sum(intensity_image)

def ROI_intensity(lb,puncta_path, r, df_filtered):
    # calculate ROI mean intensity for four channels
    roi= np.unique(lb[lb != 0])
    df_mean = pd.DataFrame(data=np.empty([len(roi),1]), columns=['roi'], dtype=object)
    df_integrated = pd.DataFrame(data=np.empty([len(roi),1]), columns=['roi'], dtype=object)
    im = zarr.open(store=zarr.N5Store(puncta_path), mode='r')
    cc = ['c0','c1','c2','c3']
    
    for c in cc:
        img = im[c +'/s2'][...]
        if c == 'c3':
            dapi=im['c2/s2'][...]
            lo=np.percentile(np.ndarray.flatten(dapi),99.5)
            bg_dapi=np.percentile(np.ndarray.flatten(dapi[dapi!=0]),1)
            bg_img=np.percentile(np.ndarray.flatten(img[img!=0]),1)
            dapi_factor=np.median((img[dapi>lo] - bg_img)/(dapi[dapi>lo] - bg_dapi))
            img = np.maximum(0, img - bg_img - dapi_factor * (dapi - bg_dapi)).astype('float32')
        elif c == 'c2':
            img = im['c2/s2'][...] 
        else:
            dapi=im['c2/s2'][...]
            lo=np.percentile(np.ndarray.flatten(dapi),99.5)
            bg_dapi=np.percentile(np.ndarray.flatten(dapi[dapi!=0]),1)
            bg_img=np.percentile(np.ndarray.flatten(img[img!=0]),1)
            dapi_factor=0
            img = np.maximum(0, img - bg_img - dapi_factor * (dapi - bg_dapi)).astype('float32') 

        lb_stat = regionprops(lb,intensity_image=img,extra_properties=(integrated_intensity,))
        for i in range(0,len(roi)):
            df_mean.loc[i, 'roi'] = lb_stat[i].label
            df_integrated.loc[i, 'roi'] = lb_stat[i].label
            df_mean.loc[i, r+'_'+c+'_mean_intensity'] = lb_stat[i].mean_intensity
            df_integrated.loc[i, r+'_'+c+'_integrated_intensity'] = lb_stat[i].integrated_intensity  
            
    df_mean.to_csv(seg_dir + r + '_mean_intensity.csv', index=False)
    df_integrated.to_csv(seg_dir + r + '_integrated_intensity.csv', index=False)
    return df_mean,df_integrated

def non_neuron_ROI(df_mean,df_filtered,r):    
    
    # calculate ROI mean intensity for four channels
    # remove low expressor of C0C3 ratio
    # remove small area 1.5*ratio
    
    print('ROI mean intensity for channels:c0-c3')
    ratio = 2
#     df_integrated = pd.read_csv(seg_dir + r + '_integrated_intensity.csv',sep=',', index_col=0)
    n,b=np.histogram(df_mean[r+'_c0_mean_intensity'].values, bins=5000)    
    thres0=b[np.argwhere(n == n.max())][0][0]*ratio
    print(thres0)
    n,b=np.histogram(df_mean[r+'_c1_mean_intensity'].values, bins=5000)    
    thres1=b[np.argwhere(n == n.max())][0][0]*ratio
    print(thres1)
    n,b=np.histogram(df_mean[r+'_c2_mean_intensity'].values, bins=5000)    
    thres2=b[np.argwhere(n == n.max())][0][0]*ratio
    print(thres2)
    n,b=np.histogram(df_mean[r+'_c3_mean_intensity'].values, bins=5000)    
    thres3=b[np.argwhere(n == n.max())][0][0]*ratio
    print(thres3)
    ROI_c0 = np.where(df_mean[r+'_c0_mean_intensity'].values < thres0)[0]
    ROI_c1 = np.where(df_mean[r+'_c1_mean_intensity'].values < thres1)[0]
    ROI_c2 = np.where(df_mean[r+'_c2_mean_intensity'].values < thres2)[0]
    ROI_c3 = np.where(df_mean[r+'_c3_mean_intensity'].values < thres3)[0]

    ROI_c0 = df_mean['roi'][ROI_c0].tolist()
    ROI_c1 = df_mean['roi'][ROI_c1].tolist()
    ROI_c2 = df_mean['roi'][ROI_c2].tolist()
    ROI_c3 = df_mean['roi'][ROI_c3].tolist()
    ROI_c3_c0 = np.array(list(set(ROI_c0).intersection(set(ROI_c3)))) # intersection？ 
    # print(ROI_c3_c0.shape[0])
    ROI_cx = ROI_c3_c0
#     ROI_cx = np.array(list(set(ROI_c3_c0).intersection(set(ROI_c2))))
    print('lowerexpressorinc0c3:' + str(len(ROI_c3_c0)))
    
    # remove small area
    n,b=np.histogram(df_filtered['area'].values, bins=5000)    
    thres=b[np.argwhere(n == n.max())][0][0]*1.5*ratio
    print('areas>'+ str(np.round(thres)))
    ROI_area =np.where(df_filtered['area'] < thres)[0]
    ROI_area = df_filtered['roi'][ROI_area].tolist()
    print('smallroi:' + str(len(ROI_area)))
    
    ROI_non_neuron = np.array(list(set(ROI_cx).union(set(ROI_area))))
    print('non_neuron_ROI:' + str(len(ROI_non_neuron))) 
    
    return ROI_non_neuron,ROI_cx,ROI_area

In [2]:
def ROI_spot(out_dir,lb,spot_dir,rr,s):
    
#     roi_dir            = out_dir + 'roi_all.csv'   # directory to file containing the ROI metadata (neuron volume, etc.)
#     spot_dir           = out_dir + 'R1_3tm50_1920/*.txt'  # directory to folder of airlocalize output (1 file/gene, txt format)
#     spotcount_dir      = out_dir + 'R1_3tm50_1920/roi_spots_all.csv'   # directory to assignedhen spots per neuron based on airlocalize (csv format)
#     output_dir         = out_dir + 'R1_3tm50_1920/Dense_spotcount.csv' # directory where output should be stored
    
    ## found spot with the roi 
    # lb = imread(lb_dir)      
    lb_id = np.unique(lb[lb != 0])
    z, y, x = lb.shape
    a = 0
#     s = [0.92,0.92,0.84]  ## voxel size in segmentation image [0.23, 0.23, 0.42] 
    count = pd.DataFrame(np.empty([len(lb_id), 0]), index=lb_id)
    fx=sorted(glob(spot_dir))     # in um
    for f in fx:
        r=os.path.basename(f).split('.')[0]
        spot=np.loadtxt(f, delimiter=',')                  
        print("Load:", f)
        # convert from physical unit to pixel unit 
        rounded_spot = np.round(spot[:, :3]/s).astype('int')    ############## every spot location in pixel
        df = pd.DataFrame(np.zeros([len(lb_id), 1]),
                          index=lb_id, columns=['count'])    ## only for count
        spot=np.append(spot, np.zeros((len(spot),1)), axis=1)  ############ every spot location in um ## add last column    
        n = len(spot)
        for i in range(0, n):                 ## lb = all the valid ROI
            if np.any(np.isnan(spot[i,:3])):  ## spot[i,:3]   ##if spot located outside of segmentation image
                print('NaN found in {} line# {}'.format(f, i+1))
            else:
                if np.any(spot[i,:3]<0):
                    a = a + 1
                else:
                    try:
                        Coord = rounded_spot[i]
                        idx = lb[Coord[2]-1, Coord[1]-1, Coord[0]-1]   # roi id                     
                        if idx > 0 and idx <= np.max(lb_id):
                            df.loc[idx, 'count'] = df.loc[idx, 'count']+1
                            spot[i,4]=idx  # add ROI number                                       
                    except Exception as e:  # outside of ROIs
                        a = a + 1

        spot_int=spot[spot[:,4]!=0]
        print(spot_int.shape)
        np.savetxt(out_dir  + 'RS-FISH/' + r +'_ROI.txt'.format(r),spot_int,delimiter=',')  
        ## found count with the roi_num  
        count.loc[:, r] = df.to_numpy() # r

    out_dir1 = out_dir + 'RS-FISH/' + rr + '_spots_all.csv'
    count.to_csv(out_dir1)
    # also save to txt.
    # spotcount=pd.read_csv(spotcount_dir,sep=',', index_col=0)
    return count

## Filter segmented ROIs for fixed round

#### load masks and images

## Later analysis for generate ROI_spots once the extracted spots has been updated

In [7]:
%%time
# import all packages
from glob import glob
from skimage.io import imread, imsave
seg_dir='E:/Maxprobe_analysis/R2_R1_3tm50/' # 
r = ['R2','R1'][1]
s = [0.92,0.92,0.84]

#RS-FISH
for i in [0,1]:
    segmentation = imread(seg_dir + 'R2' + '_filtered_mask.tif')
    spot_dir = seg_dir + 'RS-FISH/'+ r[i] + '*warped.txt'
#     spot_dir = seg_dir + 'RS-FISH/'+ r[i] + '_c3.txt'
    count = ROI_spot(seg_dir,segmentation,spot_dir,r[i],s)   
    out_dir = seg_dir + 'RS-FISH/'+ r[i] + '_allroi.csv'
    df_filtered,_ = ROI_meta(segmentation,out_dir)

Load: E:/Maxprobe_analysis/R2_R1_3tm50/RS-FISH\R1_c0_warped.txt
(127022, 5)
Load: E:/Maxprobe_analysis/R2_R1_3tm50/RS-FISH\R1_c3_warped.txt
(143752, 5)
(340, 4)
(340, 4)
CPU times: total: 49.5 s
Wall time: 51.4 s


# Exam the spot position after bigstream registration

In [23]:
# viewer = napari.view_image(data.astronaut(), rgb=True)
# napari.run()
spot_extraction = ['hAir/','RS-FISH/'][1]
spot_fix_c3_all = np.loadtxt (seg_dir + spot_extraction + 'R2_c3_ROI.txt',delimiter=',')
spot_fix_c0_all = np.loadtxt (seg_dir + spot_extraction + 'R2_c0_ROI.txt',delimiter=',')
warp_spots_c3_all = np.loadtxt (seg_dir + spot_extraction + 'R1_c3_warped_ROI.txt',delimiter=',')
warp_spots_c0_all = np.loadtxt (seg_dir + spot_extraction + 'R1_c0_warped_ROI.txt',delimiter=',')
fix_spacing = np.array([0.42,0.23,0.23])

aa = 5
print('ROI: #' + str(aa))
fix_c3 = spot_fix_c3_all[spot_fix_c3_all[:,4] == aa][:,:3]
fix_c0 = spot_fix_c0_all[spot_fix_c0_all[:,4] == aa][:,:3]
warp_c3 = warp_spots_c3_all[warp_spots_c3_all[:,4]== aa][:,:3]
warp_c0 = warp_spots_c0_all[warp_spots_c0_all[:,4]== aa][:,:3]

# change to zyx order
fix_c3 = np.transpose(np.array([fix_c3[:,2],fix_c3[:,1],fix_c3[:,0]]))
warp_c3 = np.transpose(np.array([warp_c3[:,2],warp_c3[:,1],warp_c3[:,0]]))
fix_c0 = np.transpose(np.array([fix_c0[:,2],fix_c0[:,1],fix_c0[:,0]]))
warp_c0 = np.transpose(np.array([warp_c0[:,2],warp_c0[:,1],warp_c0[:,0]]))

s=fix_c0[:,:3]/fix_spacing#convert spot um to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='fix_c0', size=3,
                  face_color='magenta',edge_color='magenta',blending='opaque')
s=warp_c0[:,:3]/fix_spacing#convert spot um to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='bigstream_c0', size=3,
                  face_color='green',edge_color='green',blending='opaque')

s=fix_c3[:,:3]/fix_spacing#convert spot um to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='fix_c3', size=3,
                  face_color='magenta',edge_color='magenta',blending='opaque')
s=warp_c3[:,:3]/fix_spacing#convert spot um to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='bigstream_c3', size=3,
                  face_color='green',edge_color='green',blending='opaque')

ROI: #5


C:\Users\zhenggang\Anaconda3\envs\easifish\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
C:\Users\zhenggang\Anaconda3\envs\easifish\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
C:\Users\zhenggang\Anaconda3\envs\easifish\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
C:\Users\zhenggang\Anaconda3\envs\easifish\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a

<Points layer 'bigstream_c3' at 0x1e7abe66a90>

### After ICP affine

In [22]:
# viewer = napari.view_image(data.astronaut(), rgb=True)
# napari.run()
Rounds = ['R2','R1']
chn = ['c3','c0']
spot_extraction = ['hAir/','RS-FISH/affine_cca/'][1]

spot_fix_c3_all = np.loadtxt (seg_dir + spot_extraction + 'ROIaffine_'+ Rounds[0] + '_' + chn[0] + '_spots.txt',delimiter=',')
spot_fix_c0_all = np.loadtxt (seg_dir + spot_extraction + 'ROIaffine_'+ Rounds[0] + '_' + chn[1] + '_spots.txt',delimiter=',')
warp_spots_c3_all = np.loadtxt (seg_dir + spot_extraction + 'ROIaffine_'+ Rounds[1] + '_' + chn[0] + '_spots.txt',delimiter=',')
warp_spots_c0_all = np.loadtxt (seg_dir + spot_extraction + 'ROIaffine_'+ Rounds[1] + '_' + chn[1] + '_spots.txt',delimiter=',')

fix_spacing = np.array([0.42,0.23,0.23])

aa = 5
print('ROI: #' + str(aa))
fix_c3 = spot_fix_c3_all[spot_fix_c3_all[:,3] == aa][:,:3]*fix_spacing
fix_c0 = spot_fix_c0_all[spot_fix_c0_all[:,3] == aa][:,:3]*fix_spacing
warp_c3 = warp_spots_c3_all[warp_spots_c3_all[:,3]== aa][:,:3]*fix_spacing
warp_c0 = warp_spots_c0_all[warp_spots_c0_all[:,3]== aa][:,:3]*fix_spacing

s=fix_c0[:,:3]/fix_spacing#convert spot um to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='fix_c0', size=3,
                  face_color='magenta',edge_color='magenta',blending='opaque')
s=warp_c0[:,:3]/fix_spacing#convert spot um to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='warp_c0', size=3,
                  face_color='green',edge_color='green',blending='opaque')

s=fix_c3[:,:3]/fix_spacing#convert spot um to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='fix_c3', size=3,
                  face_color='magenta',edge_color='magenta',blending='opaque')
s=warp_c3[:,:3]/fix_spacing#convert spot um to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='warp_c3', size=3,
                  face_color='green',edge_color='green',blending='opaque')

ROI: #5


C:\Users\zhenggang\Anaconda3\envs\easifish\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
C:\Users\zhenggang\Anaconda3\envs\easifish\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
C:\Users\zhenggang\Anaconda3\envs\easifish\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
C:\Users\zhenggang\Anaconda3\envs\easifish\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a

<Points layer 'warp_c3' at 0x1e7b61c6b50>

In [21]:
spot_fix_c3_all

array([[5.02102984e+01, 7.47849514e+02, 8.81808216e+02, 1.00000000e+00],
       [5.78701982e+01, 7.35393913e+02, 8.72666916e+02, 1.00000000e+00],
       [5.79318982e+01, 7.49952414e+02, 8.71985316e+02, 1.00000000e+00],
       ...,
       [1.03905667e+03, 1.26312072e+03, 1.07460102e+02, 1.12500000e+03],
       [1.03381707e+03, 1.26856492e+03, 1.09295002e+02, 1.12500000e+03],
       [1.03151917e+03, 1.27031922e+03, 1.17086602e+02, 1.12500000e+03]])